In [2]:
import tensorflow as tf
import collections

UNKNOWN_LABEL = "UNK"


def load_corpus(filename):
    corpus = []
    with open(filename) as infile:
        for line in infile:
            corpus.append(tf.compat.as_str(line.strip()).split())
    return corpus
            
def stream_corpus(filename):
    with open(filename) as infile:
        for line in infile:
            yield line.strip().split()


def compute_stats_and_fmap(corpus, min_shows):

    stats = {}
    for sent in corpus:
        for w in sent:
            stats[w] = stats.get(w, 0) + 1

    stats_filtered = {}
    fmap = {UNKNOWN_LABEL: 0}
    fid = 1
    for w, count in stats.iteritems():
        if count >= min_shows:
            fmap[w] = fid
            fid += 1
            stats_filtered[w] = fid
    return stats_filtered, fmap


def process_corpus(corpus, min_shows=5, add_unk=True):
    stats, fmap = compute_stats_and_fmap(corpus, min_shows)
    dataset = []
    for sent in corpus:
        current_sent = []
        for w in sent:
            if w in fmap:
                current_sent.append(fmap[w])
            else:
                if add_unk:
                    current_sent.append(fmap[UNKNOWN_LABEL])
        dataset.extend(current_sent)

    reversed_fmap = dict((fid, word) for word, fid in fmap.iteritems())
    return dataset, stats, fmap, reversed_fmap


def sent2skip(sentence, context_window, num_skips):
    """
    generate
    """


class BatchSentenceIter(object):
    def __init__(self, corpus_iter, num_skipgrams, context_window, batch_size):
        self._corpus_iter = corpus_iter # corpus_iter returns sentences
        self._batch_size = batch_size
        self._context_window = context_window
        self._span = 2 * self._context_window + 1
        
        # batch contains central words 
        # labels contains context words
        self._next_batch = np.ndarray(batch_size, dtype=np.int32)
        self._next_labels = np.ndarray((batch_size, 1), dtype=np.int32)
        self._next_batch_examples = 0
        
    def next(self):
        
        batch = self._next_batch
        labels = self._next_labels
        batch_examples = self._next_batch_examples
        
        self._next_batch = np.ndarray(batch_size, dtype=np.int32)
        self._next_labels = np.ndarray((batch_size, 1), dtype=np.int32)
        self._next_batch_examples = 0
        while batch_examples < self._batch_size:
            buf = next(self._corpus_iter)

            for (w, context) in sent2skip(buf):
                if batch_examples < batch_size:
                    self._batch[batch_examples] = w 
                    self._label[batch_examples] = context
                    batch_examples += 1
                else:
                    self._next_batch[next_batch_examples] = w 
                    self._next_label[next_batch_examples] = context
                    self._next_batch_examples += 1
            # generate skipgram pairs from sentence
        
        return self._batch, self._labels

class BatchSimpleIter(object):
    pass

In [3]:
import tensorflow as tf
import numpy as np
import math

# Create computational graph

vocabulary_size = 50000
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
    
  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
     
    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size)
    )

    
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [ ]:
batch_iter = BatchSentenceIter(corpus_iter, num_skipgrams, context_window, batch_size)
num_passes = 5

# Propagate data through graph
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_passes):
        for batch_inputs, batch_labels in batch_iter:

            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000

                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()

                for i in xrange(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in xrange(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                        print(log_str)

    final_embeddings = normalized_embeddings.eval()

    
    
    
    
    
    
    